In [1]:
# Create Enviornment 
"""
$ conda install -c conda-forge astromatic-source-extractor astromatic-swarp  # install SExtractor and SWarp (optional)
$ conda create -y --name envsfft python=3.6.6  # create Python Env
$ conda activate envsfft
$ (envsfft): pip install sfft==1.4.1  # install latest sfft via PyPI

"""

'\n$ conda install -c conda-forge astromatic-source-extractor astromatic-swarp  # install SExtractor and SWarp (optional)\n$ conda create -y --name envsfft python=3.6.6  # create Python Env\n$ conda activate envsfft\n$ (envsfft): pip install sfft==1.4.1  # install latest sfft via PyPI\n\n'

## STEP 1. specify input and output files

### A. Prepare Reference and Science images
**The image pair should be well aligned to each other.**

**However sky subtraction is NOT required in crowded field case.**

### B. Specify the output path of difference image

In [6]:
import os
import os.path as pa
from sfft.EasyCrowdedPacket import Easy_CrowdedPacket

CDIR = os.path.abspath("") # get current directory
FITS_REF = CDIR + '/input_data/ztf_001735_zg_c01_q2_refimg.resampled.mini.fits'            # reference 
FITS_SCI = CDIR + '/input_data/ztf_20180705481609_001735_zg_c01_o_q2_sciimg.mini.fits'     # science
FITS_DIFF = CDIR + '/output_data/%s.sfftdiff.fits' %(pa.basename(FITS_SCI)[:-5])          # difference

## STEP 2. setting meta-parameters

### A. BACKEND_4SUBTRACT
SFFT has two backends (CPU & GPU), setting BACKEND_4SUBTRACT = 'Numpy' ('Cupy') if you want to use CPU (GPU)

### B. CUDA_DEVICE_4SUBTRACT 
If you are using GPU (i.e., 'Cupy') backend, SFFT allow you to specify which GPU device to use by specify gpu index via CUDA_DEVICE_4SUBTRACT

### C. NUM_CPU_THREADS_4SUBTRACT
If you are using CPU (i.e., 'Numpy') backend, multiple threading is allowed for speedup and recommended number of threads is 4/8. 

### D. GAIN_KEY and SATUR_KEY 

**Gain** and **Saturation** are required in FITS header of reference and science images.

**Specifying saturation level is important for crowded case**: SFFT can temporaily mask pixels contaminated by saturations using SExtractor to eliminate their impact on the subtraction performance. Saturation Level is not necessarily very accurate, but a conservative value is recommended to make sure that all possible contaminated pixels will be masked: it is ok to set an underestimated value.

**Gain**: Gain value is not very important for crowded case, one can use a placeholder like GAIN=1.0.

In [7]:
# * computing backend and resourse 
BACKEND_4SUBTRACT = 'Numpy'     # FIXME {'Cupy', 'Numpy'}, Use 'Numpy' if you only have CPUs
CUDA_DEVICE_4SUBTRACT = '0'     # FIXME ONLY work for backend Cupy
NUM_CPU_THREADS_4SUBTRACT = 8   # FIXME ONLY work for backend Numpy

# * required info in FITS header
GAIN_KEY = 'GAIN'               # NOTE Keyword of Gain in FITS header
SATUR_KEY = 'SATURATE'          # NOTE Keyword of Saturation in FITS header

## STEP3. configurations for subtraction

### A. ForceConv
**ForceConv determines the direction of convolution, can be ['AUTO', 'REF', 'SCI']**

**'AUTO'**: convolve the image with smaller FWHM to avoid deconvolution.

**'REF'**: convolve the reference image, DIFF = SCI - Convolved REF. **(DIFF has consistent PSF and flux zero-point  with SCI)**.

**'SCI'**: convolve the science image, DIFF = Convolved SCI - REF. **(DIFF has consistent PSF and flux zero-point  with REF)**.

**Warning**: the estimation of image FWHM depends on point sources. Therefore, the lack of stars in crowded field can make the FWHM estimations not accurate. **If you already know which image has better seeing, please specify the better seeing image to be convolved instead of using 'AUTO'.**

### B. GKerHW 

Given half-width of matching kernel. E.g., GKerHW = 5, the matching kernel has a size 11 x 11.

**A rule of thumb: optimial GKerHW ~ 2 * max([FWHM_SCI, FWHM_REF])**

### C. KerHWRatio (default, 2)

Automatic half-width of matching kernel determined by FWHM. 
E.g., KerHWRatio = 2 with FWHM_REF = 3.0 and FWHM_SCI = 2.5, the matching kernel half-width will be 6 and size is 13 x 13. 

**Warning**: the estimated FWHM can be not accurate in crowded field.

**Note**: KerHWRatio will be overrided when GKerHW is not None.

### D. KerPolyOrder (default, 2)
Polynomial Order of Spatial Variation of Matching Kernel. It determines the flexibility of matching kernel across the image field. 

**KerPolyOrder = 2 is commonly a good choice in most cases.**

### E. BGPolyOrder (default, 2)
Polynomial Order of Spatial Variation of Differential Background. 

**KerPolyOrder = 2 or 3 are commonly good choices in most cases.**

### F. ConstPhotRatio (default, True)
Image subtraction needs to align the different photometric scaling of science and reference image. One have two choices for the scaling by convolution in sfft subtraction.

**Constant scaling across the image field**: setting ConstPhotRatio = True, the sfft convolution scale the flux over the image with a constant, i.e., the sum of matching kernel does not change across the field.

**Varying scaling across the image field**: setting ConstPhotRatio = False, the sfft convolution scale the flux with spatial variation following the same form determined by the parameter KerPolyOrder. e.g., says KerPolyOrder = 2, the sum of matching kernel follows a two-ordered polynomial surface across the field.

### G. PriorBanMask 
SFFT can only automatically mask saturation regions for solving the image matching. 

**One can provide additional mask for bad pixel to feed into SFFT using PriorBanMask.** 

The pixels which have True values in PriorBanMask will be temporarily masked to avoid any undesired affaction to sfft fitting.

**Note: these bad pixels will not be masked on difference image**

In [8]:
# * how to subtract
ForceConv = 'REF'               # FIXME {'AUTO', 'REF', 'SCI'}
GKerHW = None                   # FIXME given matching kernel half width
KerHWRatio = 2.0                # FIXME Ratio of kernel half width to FWHM (typically, 1.5-2.5).
KerPolyOrder = 2                # FIXME {0, 1, 2, 3}, Polynomial degree of kernel spatial variation
BGPolyOrder = 2                 # FIXME {0, 1, 2, 3}, Polynomial degree of differential background spatial variation.
ConstPhotRatio = True           # FIXME Constant photometric ratio between images?
PriorBanMask = None             # FIXME None or a boolean array with same shape of science/reference.


## step4. run the subtraction

In [14]:
PixA_DIFF, SFFTPrepDict = Easy_CrowdedPacket.ECP(FITS_REF=FITS_REF, FITS_SCI=FITS_SCI, \
    FITS_DIFF=FITS_DIFF, FITS_Solution=None, ForceConv=ForceConv, GKerHW=GKerHW, \
    KerHWRatio=KerHWRatio, KerHWLimit=(2, 20), KerPolyOrder=KerPolyOrder, BGPolyOrder=BGPolyOrder, \
    ConstPhotRatio=ConstPhotRatio, MaskSatContam=False, GAIN_KEY=GAIN_KEY, SATUR_KEY=SATUR_KEY, \
    BACK_TYPE='AUTO', BACK_VALUE=0.0, BACK_SIZE=64, BACK_FILTERSIZE=3, DETECT_THRESH=5.0, \
    DETECT_MINAREA=5, DETECT_MAXAREA=0, DEBLEND_MINCONT=0.005, BACKPHOTO_TYPE='LOCAL', \
    ONLY_FLAGS=None, BoundarySIZE=0.0, BACK_SIZE_SUPER=128, StarExt_iter=2, PriorBanMask=PriorBanMask, \
    BACKEND_4SUBTRACT=BACKEND_4SUBTRACT, CUDA_DEVICE_4SUBTRACT=CUDA_DEVICE_4SUBTRACT, \
    NUM_CPU_THREADS_4SUBTRACT=NUM_CPU_THREADS_4SUBTRACT)[:2]
print('MeLOn CheckPoint: TEST FOR CROWDED-FLAVOR-SFFT SUBTRACTION DONE!\n')


MeLOn CheckPoint: TRIGGER Crowded-Flavor Auto Preprocessing!

MeLOn CheckPoint [ztf_001735_zg_c01_q2_refimg.resampled.mini.fits]: Run Python Wrapper of SExtractor!
MeLOn CheckPoint [ztf_001735_zg_c01_q2_refimg.resampled.mini.fits]: SExtractor uses GAIN = [6.2] from keyword [GAIN]!
MeLOn CheckPoint [ztf_001735_zg_c01_q2_refimg.resampled.mini.fits]: SExtractor uses SATURATION = [50581.142] from keyword [SATURATE]!
MeLOn CheckPoint [ztf_001735_zg_c01_q2_refimg.resampled.mini.fits]: SExtractor found [532] sources!
MeLOn CheckPoint [ztf_001735_zg_c01_q2_refimg.resampled.mini.fits]: PYSEx output catalog contains [532] sources!

MeLOn CheckPoint [ztf_20180705481609_001735_zg_c01_o_q2_sciimg.mini.fits]: Run Python Wrapper of SExtractor!
MeLOn CheckPoint [ztf_20180705481609_001735_zg_c01_o_q2_sciimg.mini.fits]: SExtractor uses GAIN = [6.2] from keyword [GAIN]!
MeLOn CheckPoint [ztf_20180705481609_001735_zg_c01_o_q2_sciimg.mini.fits]: SExtractor uses SATURATION = [48866.145] from keyword [SATURA

In [15]:
from astropy.io import fits

# 1. which side is convolved in image subtraction?
#    look into the header of difference image

CONVD = fits.getheader(FITS_DIFF, ext=0)['CONVD']
print('MeLOn CheckPoint: [%s] is convolved in sfft subtraction' %CONVD)
if CONVD == 'SCI': print('* DIFF = Convolve(SCI) - REF has PSF and zero-point aligned with [REF]!')
if CONVD == 'REF': print('* DIFF = SCI - Convolve(REF) has PSF and zero-point aligned with [SCI]!')
    
# 2. FWHM Estimations (May not accurate for crowded field where point sources are limited)
FWHM_REF = SFFTPrepDict['FWHM_REF']
FWHM_SCI = SFFTPrepDict['FWHM_SCI']
print('\nMeLOn CheckPoint: FWHM_REF = [%.2f pix]' %FWHM_REF)
print('\nMeLOn CheckPoint: FWHM_SCI = [%.2f pix]' %FWHM_SCI)


MeLOn CheckPoint: [REF] is convolved in sfft subtraction
* DIFF = SCI - Convolve(REF) has PSF and zero-point aligned with [SCI]!

MeLOn CheckPoint: FWHM_REF = [2.03 pix]

MeLOn CheckPoint: FWHM_SCI = [1.77 pix]
